# DATA QUALITY
---



## 0. IMPORT DATA

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, sum

spark = SparkSession.builder.getOrCreate()

In [2]:
git_commit = spark.read.csv('../data/GIT_COMMITS.csv',header=True)
changes = spark.read.csv('../data/GIT_COMMITS_CHANGES.csv',header=True)
ref_miner = spark.read.csv('../data/REFACTORING_MINER.csv',header=True)

## 1. GIT_COMMITS

In [3]:
git_commit = git_commit.drop('PROJECT_ID')
git_commit = git_commit.drop('AUTHOR')
git_commit = git_commit.drop('AUTHOR_DATE')
git_commit = git_commit.drop('AUTHOR_TIMEZONE')
git_commit = git_commit.drop('COMMITTER')
git_commit = git_commit.drop('COMMITER_DATE')
git_commit = git_commit.drop('COMMITTER_TIMEZONE')
git_commit = git_commit.drop('BRANCHES')
git_commit = git_commit.drop('IN_MAIN_BRANCH')
git_commit = git_commit.drop('MERGE')

#### Check for duplicate values.

In [4]:
print("DF has :",git_commit.count(),"rows")
print("Distinct DF has :",git_commit.distinct().count(),"rows")

DF has : 193152 rows
Distinct DF has : 156318 rows


In [5]:
print("There are :",git_commit.count()-git_commit.distinct().count(),"duped rows")

There are : 36834 duped rows


The duplicates could be caused by the column dropping. To check so we will see the number of dupped rows of the original DF and the column filtered one.

In [6]:
git_commit_og = spark.read.csv('../data/GIT_COMMITS.csv',header=True)

In [7]:
print("There are :",git_commit_og.count()-git_commit_og.distinct().count(),"duped rows in the original DF")
print("There are :",git_commit.count()-git_commit.distinct().count(),"duped rows in the new DF")

There are : 13288 duped rows in the original DF
There are : 36834 duped rows in the new DF


Check the duped lines for possible patterns.

In [8]:
dup = git_commit.groupBy(["COMMIT_HASH","COMMIT_MESSAGE","PARENTS"]).count().filter("count > 1")
dup.show()

+--------------------+--------------------+-------+-----+
|         COMMIT_HASH|      COMMIT_MESSAGE|PARENTS|count|
+--------------------+--------------------+-------+-----+
|    Sebastian Bazley|2014-08-28T23:09:12Z|   null|    2|
| the deprecated c...|Expression}Factor...|   null|    2|
|                  so|                null|   null|    3|
|       Henri Yandell|2006-01-24T14:29:48Z|   null|   17|
|     COLLECTIONS-253|     COLLECTIONS-293|   null|   15|
| from Dimiter Dim...|                null|   null|    2|
|   from Nathan Beyer|                null|   null|    3|
| 11 Sep 2009) | 1...|                null|   null|   14|
|                  it|                null|   null|    4|
|                 Get| and IterableGet ...|   null|    4|
|    Sebastian Bazley|2012-03-07T00:08:20Z|   null|    2|
|       Henri Yandell|2007-07-29T20:14:22Z|   null|   16|
|                   a|                null|   null|    2|
|    Sebastian Bazley|2011-09-10T14:06:53Z|   null|    2|
|     Niall Pe

#### Check for missing values.

In [9]:
git_commit.select([count(when(col(c).isNull(), c)).alias(c) for c in git_commit.columns]).show()

+-----------+--------------+-------+
|COMMIT_HASH|COMMIT_MESSAGE|PARENTS|
+-----------+--------------+-------+
|      34799|         38629| 184795|
+-----------+--------------+-------+



In [10]:
git_commit.filter(col("COMMIT_HASH").isNull()).show()

+-----------+--------------+-------+
|COMMIT_HASH|COMMIT_MESSAGE|PARENTS|
+-----------+--------------+-------+
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
|       null|          null|   null|
+-----------+--------------+-------+
only showing top 20 rows



As "COMMIT_HASH" is our join key we can't use rows with a missing hash. The same for "COMMIT_MESSAGE" as we need to use it for the model and a null message is not useful.

In [11]:
git_commit=git_commit.na.drop(subset=["COMMIT_HASH"])
git_commit=git_commit.na.drop(subset=["COMMIT_MESSAGE"])

In [12]:
git_commit.select([count(when(col(c).isNull(), c)).alias(c) for c in git_commit.columns]).show()

+-----------+--------------+-------+
|COMMIT_HASH|COMMIT_MESSAGE|PARENTS|
+-----------+--------------+-------+
|          0|             0| 146132|
+-----------+--------------+-------+



## 2. GIT_COMMIT_CHANGES

In [13]:
changes = changes.drop('PROJECT_ID')
changes = changes.drop('DATE')
changes = changes.drop('COMMITTER_ID')

#### Check for duplicate rows.

In [14]:
print("DF has :",changes.count(),"rows")
print("Distinct DF has :",changes.distinct().count(),"rows")

DF has : 890223 rows
Distinct DF has : 817121 rows


In [15]:
print("There are :",changes.count()-changes.distinct().count(),"duped rows")

There are : 73102 duped rows


The duplicates could be caused by the column dropping. To check so we will see the number of dupped rows of the original DF and the column filtered one.

In [16]:
changes_og = spark.read.csv('../data/GIT_COMMITS_CHANGES.csv',header=True)

In [17]:
print("There are :",changes_og.count()-changes_og.distinct().count(),"duped rows in the original DF")
print("There are :",changes.count()-changes.distinct().count(),"duped rows in the new DF")

There are : 72476 duped rows in the original DF
There are : 73102 duped rows in the new DF


Check the duped lines for possible patterns.

In [18]:
dup = changes.groupBy(["FILE","COMMIT_HASH","LINES_ADDED","LINES_REMOVED","NOTE"]).count().filter("count > 1")
dup.show()

+--------------------+--------------------+-----------+-------------+--------------------+-----+
|                FILE|         COMMIT_HASH|LINES_ADDED|LINES_REMOVED|                NOTE|count|
+--------------------+--------------------+-----------+-------------+--------------------+-----+
|  testing-1.0.tar.gz|dee0d5a300ee0ba02...|          0|            0|Merge from archiv...|    2|
|get-relocated-art...|0314ca95cbd058011...|          0|            0|More directory mo...|    2|
|newversion-artifa...|a05f42cc0ec53b094...|          0|            0|restructure accor...|    2|
|v4-timestamped-sn...|58da8544b7419af03...|         11|            0|Branch for metada...|    2|
|             pom.xml|58da8544b7419af03...|         99|            0|Branch for metada...|    2|
|             pom.xml|58da8544b7419af03...|         80|            0|Branch for metada...|    2|
|             pom.xml|58b08d0094b8f862a...|          1|            1|bump to 1.4-M3-SN...|   69|
|test-not-updated-...|24f005ce

#### Check for missing values

In [19]:
changes.select([count(when(col(c).isNull(), c)).alias(c) for c in changes.columns]).show()

+-----+-----------+-----------+-------------+-----+
| FILE|COMMIT_HASH|LINES_ADDED|LINES_REMOVED| NOTE|
+-----+-----------+-----------+-------------+-----+
|32481|      32481|      32481|        32481|32481|
+-----+-----------+-----------+-------------+-----+



As we can see the rows that are missing are comprised completely by null values, we will remove them.

In [20]:
changes=changes.na.drop("all")

In [21]:
changes.select([count(when(col(c).isNull(), c)).alias(c) for c in changes.columns]).show()

+----+-----------+-----------+-------------+----+
|FILE|COMMIT_HASH|LINES_ADDED|LINES_REMOVED|NOTE|
+----+-----------+-----------+-------------+----+
|   0|          0|          0|            0|   0|
+----+-----------+-----------+-------------+----+



## 3. REFACTORING_MINER

In [22]:
ref_miner = ref_miner.drop('PROJECT_ID')

#### Check for duplicated rows.

In [23]:
print("DF has :",ref_miner.count(),"rows")
print("Distinct DF has :",ref_miner.distinct().count(),"rows")

DF has : 37226 rows
Distinct DF has : 37100 rows


In [24]:
print("There are :",ref_miner.count()-ref_miner.distinct().count(),"duped rows")

There are : 126 duped rows


The duplicates could be caused by the column dropping. To check so we will see the number of dupped rows of the original DF and the column filtered one.

In [25]:
ref_miner_og = spark.read.csv('../data/REFACTORING_MINER.csv',header=True)

In [26]:
print("There are :",ref_miner_og.count()-ref_miner_og.distinct().count(),"duped rows in the original DF")
print("There are :",ref_miner.count()-ref_miner.distinct().count(),"duped rows in the new DF")

There are : 126 duped rows in the original DF
There are : 126 duped rows in the new DF


Check the duped lines for possible patterns.

In [27]:
dup = ref_miner.groupBy(["COMMIT_HASH","REFACTORING_TYPE","REFACTORING_DETAIL"]).count().filter("count > 1")
dup.show()

+--------------------+-------------------+--------------------+-----+
|         COMMIT_HASH|   REFACTORING_TYPE|  REFACTORING_DETAIL|count|
+--------------------+-------------------+--------------------+-----+
|64bf00a0317c3c8a0...|  Pull Up Attribute|Pull Up Attribute...|    3|
|f6ec5a7e86b3d5b9d...|  Pull Up Attribute|Pull Up Attribute...|    3|
|f56609efad6393b10...|     Move Attribute|Move Attribute	pr...|    3|
|316f95271bc1c676b...|     Move Attribute|Move Attribute	pu...|    3|
|f56609efad6393b10...|     Move Attribute|Move Attribute	pr...|    3|
|f56609efad6393b10...|     Move Attribute|Move Attribute	pr...|    3|
|e26eac7e06325ef51...|     Move Attribute|Move Attribute	pr...|    3|
|4b9ef913f999b1d4f...|Push Down Attribute|Push Down Attribu...|    3|
|f6ec5a7e86b3d5b9d...|  Pull Up Attribute|Pull Up Attribute...|    3|
|ff06852acdea7695d...|Push Down Attribute|Push Down Attribu...|    3|
|423f26e7564f3e2c0...|  Pull Up Attribute|Pull Up Attribute...|    3|
|f56609efad6393b10..

In [28]:
dup.summary().show()

+-------+--------------------+-------------------+--------------------+-----+
|summary|         COMMIT_HASH|   REFACTORING_TYPE|  REFACTORING_DETAIL|count|
+-------+--------------------+-------------------+--------------------+-----+
|  count|                  63|                 63|                  63|   63|
|   mean|                null|               null|                null|  3.0|
| stddev|                null|               null|                null|  0.0|
|    min|0a1b1d317f0dfcd8d...|     Move Attribute|Move Attribute	pa...|    3|
|    25%|                null|               null|                null|    3|
|    50%|                null|               null|                null|    3|
|    75%|                null|               null|                null|    3|
|    max|ff06852acdea7695d...|Push Down Attribute|Push Down Attribu...|    3|
+-------+--------------------+-------------------+--------------------+-----+



In this case the count number seems to be always 3. There seems to be no explanation.

#### Check for duplicate values.

In [29]:
ref_miner.select([count(when(col(c).isNull(), c)).alias(c) for c in ref_miner.columns]).show()

+-----------+----------------+------------------+
|COMMIT_HASH|REFACTORING_TYPE|REFACTORING_DETAIL|
+-----------+----------------+------------------+
|          1|               1|                 1|
+-----------+----------------+------------------+



In [30]:
ref_miner.filter(col("COMMIT_HASH").isNull()).show()

+-----------+----------------+------------------+
|COMMIT_HASH|REFACTORING_TYPE|REFACTORING_DETAIL|
+-----------+----------------+------------------+
|       null|            null|              null|
+-----------+----------------+------------------+



In [31]:
ref_miner = ref_miner.na.drop("all")

In [32]:
ref_miner.select([count(when(col(c).isNull(), c)).alias(c) for c in ref_miner.columns]).show()

+-----------+----------------+------------------+
|COMMIT_HASH|REFACTORING_TYPE|REFACTORING_DETAIL|
+-----------+----------------+------------------+
|          0|               0|                 0|
+-----------+----------------+------------------+

